## 本章主要讲解以下内容：
- 介绍`PyTorch`的基本数据结构`Tensor`
- `Tensor`的操作
- 和`NumPy`的多维数组的交互
- `GPU`加快计算速度

进行机器学习和深度学习项目时，第一步是将外部输入转化成计算机表示，如浮点数等等，`PyTorch`提供了很好用的`Tensor`

# Tensor fundamentals

`Python`程序经常会处理矢量数据，通常情况下，`Python`使用列表存储,但是这不是最优方案，因为：
- `Python`中的`Numbers`是功能齐全的对象，`Python`将数字和属性打包成一个对象存储，数据量大时，会变得没有效率
- `Python`中的`List`存储对象集合，没有直接的向量点积操作，而且`list`是一维的，虽然可以列表的列表来表示多维，但是不高效
- `Python`语言相较于底层语言来说速度较慢

In [4]:
import torch
a = torch.ones(3)    #构建一个含有三个1的向量
a

tensor([1., 1., 1.])

In [5]:
a[1]    #访问向量的第二个量

tensor(1.)

In [6]:
float(a[1])    #可以进行强制类型转换

1.0

In [7]:
a[2] = 2.0    #改变第三个分量
a

tensor([1., 1., 2.])

以上操作虽然看上去和`list`没有太大区别，但是底层实现差别很大，`list`在内存中存储的是一个个对象，`tensor`存储的是数值.
构建`tensor`时，可以传入一个列表：

In [8]:
points = torch.tensor([1.0, 4.0, 2.0, 1.0, 3.0, 5.0])
points

tensor([1., 4., 2., 1., 3., 5.])

In [10]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]])
points,points.shape

(tensor([[1., 4.],
         [2., 1.],
         [3., 5.]]), torch.Size([3, 2]))

也可以初始化一个固定结构的`tensor`，然后往里面填入数据:

In [12]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [13]:
points[0, 1] = 1.0
points

tensor([[0., 1.],
        [0., 0.],
        [0., 0.]])

In [14]:
points[0]    #得到第一行的tensor

tensor([0., 1.])

### **注意**

上面返回第一行的`tensor`，并不是重新分配内存生成新的`tensor`，而是返回原来`tensor`第一行的`view`，这样数据量大的话，比较有效率

# Tensors and storages

`tensor`进行存储时，`value`存储在一段连续的内存中，由`torch.Storage`实例管理,`tensor`只是这一维存储空间中数据的`view`而已，不同的解读方法会产生不同的`tensor`

In [15]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]])
points.storage()

 1.0
 4.0
 2.0
 1.0
 3.0
 5.0
[torch.FloatStorage of size 6]

这里，虽然points是一个`3*2`的`tensor`，但是在存储空间中，是一个一维的存储方式

In [16]:
points_storage = points.storage()
points_storage[0]

1.0

In [17]:
points.storage()[1]

4.0

In [18]:
points_storage[0] = 2.0   #改变存储空间中的值，points发生改变
points

tensor([[2., 4.],
        [2., 1.],
        [3., 5.]])

了解了`tensor`数据在内存中的存储方式，可以在你需要优化`pytorch`代码时发挥作用

# Size, storage offset, and strides

# Numeric types

# Indexing tensors 

# NumPy interoperability

# Serializing tensors

# Moving tensors to the GPU

# The tensor API